In [1]:
# Download data using overpass API

In [1]:
import overpy
import json
from helpers.ways_converter import ways2poly
from shapely.geometry import Polygon, MultiPolygon, Point
import geopandas as gpd

In [2]:
api = overpy.Overpass() 
# Use swiss instance of the API
api.default_url="https://overpass.osm.ch/api/interpreter"

In [3]:
results_train = api.query("""
            [out:json];
            area["ISO3166-1"="CH"]->.searchArea;
            (
            // Close to a train track?
            way["railway"="rail"]["usage"="main"](area.searchArea);
            way["railway"="rail"]["usage"="branch"](area.searchArea);
            
            // Close to metro rails?
            way["railway"="subway"](area.searchArea);
            way["railway"="subway_entrance"](area.searchArea);
            
            // Consider other "alternative" rail types, like light rail and funicular
            way["railway"="light_rail"](area.searchArea);
            way["railway"="funicular"](area.searchArea);
            way["railway"="narrow_gauge"](area.searchArea);
            way["railway"="monorail"](area.searchArea);
            
            // Just for fun: miniature rail
            way["railway"="miniature"](area.searchArea);
            
            // Consider cable cars etc as "trains" as well
            way["aerialway"](area.searchArea);
                    
            );
            out body;
            >;
            out skel qt;
                    """)

In [4]:
results_tram = api.query("""
            [out:json];
            area["ISO3166-1"="CH"]->.searchArea;
            (
            // Close to tram rails?
            way["railway"="tram"](area.searchArea);
            rel["route"="tram"](area.searchArea);
            );
            out body;
            >;
            out skel qt;
                    """)

In [5]:
# List all swiss cantons in ISO 3166-2 format
# https://en.wikipedia.org/wiki/ISO_3166-2:CH
cantons = ["AG", "AI", "AR", "BE", "BL", "BS", "FR", "GE", "GL", "GR", "JU", "LU", "NE", "NW", "OW", "SG", "SH", "SO", "SZ", "TG", "TI", "UR", "VD", "VS", "ZG", "ZH"]

In [6]:
buses_list = []

for canton in cantons:
        res = api.query(f"""
            [out:json];
            area["ISO3166-2"="CH-{canton}"]->.searchArea;
            (
                    
            // Close to bus line?
            rel["route"="trolleybus"](area.searchArea);
            rel["route"="bus"](area.searchArea);
            );
            out body;
            >;
            out skel qt;
                    """)
        buses_list.append(res)
        print(f"Finished {canton}")

Finished AG
Finished AI
Finished AR
Finished BE
Finished BL
Finished BS
Finished FR
Finished GE
Finished GL
Finished GR
Finished JU
Finished LU
Finished NE
Finished NW
Finished OW
Finished SG
Finished SH
Finished SO
Finished SZ
Finished TG
Finished TI
Finished UR
Finished VD
Finished VS
Finished ZG
Finished ZH


In [7]:
# Merge buses into one object
results_buses = overpy.Result()

for res in buses_list:
    results_buses.expand(res)

In [8]:
results_lakes = api.query("""
        [out:json];
        area["ISO3166-1"="CH"]->.searchArea;
        (
                
        // Lakes
        relation["water"="lake"](area.searchArea);
        relation["water"="reservoir"](area.searchArea);
        );
        out body;
        >;
        out skel qt;
                """)

In [9]:
log = False
# build polygons from the lakes relations
lake_boundaries = {}
for lake in results_lakes.relations:
    k = lake.id
    # Convert to list of ways
    ways = [way for way in results_lakes.ways if way.id in [m.ref for m in lake.members]]

    polys, incmp = ways2poly(ways)
    lake_boundaries[k] = {'polygons': polys, 'incomplete': incmp}

    
    if len(polys) > 0 and len(incmp) == 0:
        outcome = 'OK'
    else:
        outcome = 'ERROR'

    if log:
        print("{}: {:>2} polygons, {:>2} incomplete ({})".format(
            k, len(polys), len(incmp), outcome))

    # I only care about complete polygons, but you could process incomplete
    # ones as (Multi)LineString if needed
    if outcome == 'OK':
        lake_boundaries[k]['shape'] = MultiPolygon(
            [Polygon([(n.lon, n.lat) for n in p]) for p in polys])

In [10]:
# Load the geojson of switzerland
ch = gpd.read_file("data/switzerland.geojson")

In [11]:
# Clean results_train to keep only the points in ch, and then remove the ways with less than 2 points
train_points = []
for way in results_train.ways:
    points = [(node.lon, node.lat) for node in way.nodes if ch.contains(Point(float(node.lon), float(node.lat))).values[0]]
    if len(points) > 1:
        train_points.append(points)

In [12]:
# Clean results_tram to keep only the points in ch, and then remove the ways with less than 2 points
tram_points = []
for way in results_tram.ways:
    points = [(node.lon, node.lat) for node in way.nodes if ch.contains(Point(float(node.lon), float(node.lat))).values[0]]
    if len(points) > 1:
        tram_points.append(points)

In [13]:
# Clean results_bus to keep only the points in ch, and then remove the ways with less than 2 points
bus_points = []
for way in results_buses.ways:
    points = [(node.lon, node.lat) for node in way.nodes if ch.contains(Point(float(node.lon), float(node.lat))).values[0]]
    if len(points) > 1:
        bus_points.append(points)

In [14]:
# Build geojson object from the results
geojson = {
    "type": "FeatureCollection",
    "features": []
}

# Add train lines to geojson, but keep only the nodes inside switzerland
for way in train_points:
    geojson["features"].append({
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": [[float(node[0]), float(node[1])] for node in way]
        },
        "properties": {
            "type": "train"
        }
    })

for way in tram_points:
    geojson["features"].append({
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": [[float(node[0]), float(node[1])] for node in way]
        },
        "properties": {
            "type": "tram"
        }
    })

for way in bus_points:
    geojson["features"].append({
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": [[float(node[0]), float(node[1])] for node in way]
        },
        "properties": {
            "type": "bus"
        }
    })

# Append lakes to geojson as multipolygons
for k, v in lake_boundaries.items():
    geojson['features'].append({
        "type": "Feature",
        "geometry": {
            "type": "Polygon",
            "coordinates": [[(float(n.lon), float(n.lat)) for n in poly] for poly in v['polygons']]
        },
        "properties": {
            "type": "lake",
        }
    })


In [15]:
with open("./results/swiss_map_mot_type.geojson", "w") as f:
    json.dump(geojson, f)

In [16]:
# Reduce the size of the file

# 1. Reduce precision of coordinates, keep only 5 after the comma
geojson["features"] = [
    {
        "type": "Feature",
        "geometry": {
            "type": feature["geometry"]["type"],
            "coordinates": [[round(x, 5) if isinstance(x, float) else (round(float(x[0]), 5), round(float(x[1]), 5)) 
                             for x in coord] for coord in feature["geometry"]["coordinates"]]
        },
        "properties": feature["properties"]
    }
    for feature in geojson["features"]
]

In [17]:
# Save to compare size
with open("./results/swiss_map_mot_type_compressed.geojson", "w") as f:
    json.dump(geojson, f)

In [18]:
# Load in geopandas
gdf = gpd.read_file("./results/swiss_map_mot_type_compressed.geojson")

In [19]:
# Simplify the geometries
gdf["geometry"] = gdf["geometry"].simplify(tolerance=0.0001) # 0.0001 degrees tolerance, it's about 11.1 m!

In [20]:
# Save to geojson again
gdf.to_file("./results/swiss_map_mot_type_simplified.geojson", driver="GeoJSON")

In [21]:
# Reload as json object
with open("./results/swiss_map_mot_type_simplified.geojson", "r") as f:
    geojson_simplified = json.load(f)

In [22]:
# Transform geojson_simplified to dictionary of LineStrings and Polygons
trains = []
trams = []
buses = []
lakes = []

for feature in geojson_simplified["features"]:
    if feature["properties"]["type"] == "train":
        trains.append(feature["geometry"]["coordinates"])
    elif feature["properties"]["type"] == "tram":
        trams.append(feature["geometry"]["coordinates"])
    elif feature["properties"]["type"] == "bus":
        buses.append(feature["geometry"]["coordinates"])
    elif feature["properties"]["type"] == "lake":
        lakes.append(feature["geometry"]["coordinates"])

In [23]:
# Svae all these as json, with less information than the geojson
with open("./results/swiss_mots_small.json", "w") as f:
    json.dump({
        "trains": trains,
        "trams": trams,
        "buses": buses,
        "lakes": lakes
    }, f)

In [24]:
# Convert to the format {"trains": [{"p": ["{x: 1, y: 2}", "{x: 2, y: 3}", ...]}, ...]}
with open("./results/swiss_mots_small.json", "r") as f:
    data = json.load(f)

In [25]:
# Save the data as strings, which can then be directly put in the database as strings (without quotes, so need to use json lenient parser)
data = {
    "trains": [{"p": [{"x": x, "y": y} for x, y in line]} for line in data["trains"]],
    "trams": [{"p": [{"x": x, "y": y} for x, y in line]} for line in data["trams"]],
    "buses": [{"p": [{"x": x, "y": y} for x, y in line]} for line in data["buses"]],
    "lakes": [{"r": [{"p": [{"x": x, "y": y} for x, y in ring]} for ring in poly]} for poly in data["lakes"]]
}

In [26]:
# Save as json
with open("./results/swiss_mots.json", "w") as f:
    json.dump(data, f)

In [27]:
# Get minX, minY, maxX, maxY from each element inside a list [{"x"": 1, "y": 2}, ...]
def get_bbox(points):
    min_x = min([point["x"] for point in points])
    min_y = min([point["y"] for point in points])
    max_x = max([point["x"] for point in points])
    max_y = max([point["y"] for point in points])
    return str(min_x)+","+str(min_y)+","+str(max_x)+","+str(max_y)

In [28]:
# Create a dictionary with the names of the vehicles in the json file and in the SQL database
vehicles = {
    "trains": "TRAIN",
    "trams": "TRAM",
    "buses": "BUS",
    "lakes": "LAKE"
}

In [35]:
batch_size = 400 # Create batches of 400 queries each -- There is a limit of 500 unions, but we take 400 just to be on the safe side
query = ""
# Delete all the data from the spatialdata table, before inserting the new data
query+= f"""
DELETE FROM SpatialData;
"""

for k,v in vehicles.items():
    # Transform this to SQL queries to insert in the database, with VEHICLE, GEOMETRY in json format, and the bounding box of each element
    # so minX, minY, maxX, maxY, by getting the id of the vehicle by its name first (SELECT id FROM Vehicles WHERE name = "TRAIN";) and
    # saving that as a variable

    tq = f"""
CREATE TEMPORARY TABLE temp_{k}_id AS
SELECT id FROM OSMVehicles WHERE name = '{v}' LIMIT 1;
    """

    # Create a query for each train line -- Using this type of query, in batches, 
    # accelerates the insertion more than 10x (from 3.5s to 0.3s)
    if k == "lakes":
        t = ["SELECT tv.id,'{}',{} FROM temp_{}_id tv".format(str(poly['r']).replace('\'', '\"'), get_bbox(poly['r'][0]['p']), k) for poly in data[k]]
    else:
        t = ["SELECT tv.id,'{}',{} FROM temp_{}_id tv".format(str(way['p']).replace('\'', '\"'), get_bbox(way['p']), k) for way in data[k]]

    # Divide data in batches and create a query for each batch
    batches = [t[i:i+batch_size] for i in range(0, len(t), batch_size)]
    
    # Build a query for each batch
    queries = ["INSERT INTO SpatialData (vehicle, geometry, minX, minY, maxX, maxY)" + " UNION ALL ".join(b)+";" for b in batches]

    tq += " ".join(queries)

    query += tq

In [36]:
# Save to file, named as year_month_osm_db.sql
import datetime
with open(f"./results/{datetime.datetime.now().strftime('%Y_%m')}_osm_db.sql", "w") as f:
    f.write(query)

# How to update osm_db

1. Run this script, which generates a file in results/ named `{year}_{month}_osm_db.sql`. This file contains sql calls to populate the content of the local database (in the app) with all the railways, tram lines, buses, and lakes of Switzerland.
2. Test the generated file with `test_sqlite.ipynb`
3. Once passed the test, go to the source code of `swice_server`and:
    - Copy the generated `{year}_{month}_osm_db.sql` file in the folder `app/osm_data``
    - Open the `app/api/views.py`file and edit the `osm_get_latest_version` function to return the string `"{year}_{month}"`corresponding the the just uploaded file
    - Commit and push the changes
    - Run the github action `Publish to the production server`in the `swice_app`repository
